In [11]:
import geopandas as gpd
import fiona 
import pandas as pd
from bs4 import BeautifulSoup
import shapely
import re
from shapely.geometry import Polygon

In [12]:
def extract_border(df):
    border = df.copy()
    border['new_column'] = 0
    border = border.dissolve(by='new_column')
    border = border[['geometry']]
    border['geometry'] = border['geometry'].buffer(0.000001)
    border.index.names = ['ID']
    
    return border

def parse_html(row, dic='None'):
    html_str = row['Description']
    parsed_html = BeautifulSoup(html_str)
    array = parsed_html.body.find_all('td')
    array = array[2:]
    array = [clean_html(str(tag)) for tag in array]
    dic = dict(array[i:i+2] for i in range(0, len(array), 2))
    if not(dic['FID']):
        print(dic)
    return pd.Series(dic)
    
def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [13]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
quadras_df = gpd.read_file('./data/BASES/quadras/quadras.kml')

# Drop Z dimension of polygons that occurs often in kml 
quadras_df.geometry = quadras_df.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))

In [14]:
quadras_df = quadras_df.join(quadras_df.apply(parse_html, axis=1))
quadras_df = quadras_df.drop('Description', 1)
quadras_df = quadras_df[['FID', 'SETOR', 'geometry']]
quadras_df

,FID,SETOR,geometry
0,588,5,"MULTIPOLYGON (((-34.83164 -7.11401, -34.83224 ..."
1,637,5,"MULTIPOLYGON (((-34.82960 -7.11734, -34.82972 ..."
2,648,6,"MULTIPOLYGON (((-34.83025 -7.12070, -34.83021 ..."
3,649,6,"MULTIPOLYGON (((-34.82909 -7.11961, -34.82920 ..."
4,650,6,"MULTIPOLYGON (((-34.82935 -7.12029, -34.82951 ..."
...,...,...,...
497,1773,15,"MULTIPOLYGON (((-34.86004 -7.11769, -34.86042 ..."
498,1777,15,"MULTIPOLYGON (((-34.85339 -7.11878, -34.85339 ..."
499,1778,15,"MULTIPOLYGON (((-34.85665 -7.11868, -34.85680 ..."
500,1779,15,"MULTIPOLYGON (((-34.85815 -7.11869, -34.85839 ..."


In [15]:
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores[['CD_GEOCODI', 'NM_BAIRRO', 'geometry']]

In [16]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')
bairros['geometry'] = bairros['geometry'].buffer(0.0000001)

In [17]:
bairros_interesse_maior = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
bairros_interesse_menor = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar']

setores_recorte_maior = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse_maior)]
setores_recorte_menor = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse_menor)]

In [18]:
limite_recorte_maior = extract_border(setores_recorte_maior)
limite_recorte_menor = extract_border(setores_recorte_menor)
limite_municipio = extract_border(bairros)
limite_municipio['geometry'][0] = Polygon(limite_municipio['geometry'][0].exterior)

In [19]:
setores.to_file("./data/BASES/setores/setores.shp")
bairros.to_file("./data/BASES/bairros/bairros.shp")
limite_recorte_maior.to_file("./data/BASES/limite_recorte/limite_recorte_maior.shp")
limite_recorte_menor.to_file("./data/BASES/limite_recorte/limite_recorte_menor.shp")
limite_municipio.to_file("./data/BASES/limite_municipio/limite_municipio.shp")
quadras_df.to_file("./data/BASES/quadras/quadras.shp")